# Accessing Chicago Data with SQLite and Python
The datasets used in this project are as follows:
- **ChicagoCensusData.csv**: Socioeconomic data for the years 2008-2012. The data set and more information on it can be found [here.](https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2/about_data?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvo_campaign=000026UJ&cvosrc=email.Newsletter.M12345678&utm_content=000026UJ&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_term=10006555 "Census Data")
- **ChicagoPublicSchool**: 2011-2012 data on school performance levels used to create report cards. More information can be found on the [website](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t/about_data?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvo_campaign=000026UJ&cvosrc=email.Newsletter.M12345678&utm_content=000026UJ&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_term=10006555 "Chicago Schools")
- **ChicagoCrimeData**: the data reflects reports instances of crime from the year 2021 and is publicly available [here](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/about_data?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvo_campaign=000026UJ&cvosrc=email.Newsletter.M12345678&utm_content=000026UJ&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_term=10006555)

To get started with the project, the required packages would first be imported. 

In [1]:
import pandas as pd 
import prettytable 
import sqlite3 

prettytable.DEFAULT = 'DEFAULT'

### Store The Data in Tables of a DB
To do this, first establish a connection to a db. Then load the data a pandas dataframe and create the respective tables for the data in the database.

In [2]:
# Create a db connection 
connection = sqlite3.connect('CHICAGO.db')
cursor = connection.cursor()

# Load the data sets to dataframes
census_data = pd.read_csv('file_csv/ChicagoCensusData.csv')
chicago_crime_data = pd.read_csv('file_csv/ChicagoCrimeData.csv')
school_data = pd.read_csv('file_csv/ChicagoPublicSchools.csv')

# Load the dataframes to the database
census_data.to_sql('CENSUS_DATA', connection, if_exists='replace', index=False)
chicago_crime_data.to_sql('CHICAGO_CRIME_DATA', connection, if_exists='replace', index=False)
school_data.to_sql('CHICAGO_PUBLIC_SCHOOLS', connection, if_exists='replace', index=False)

566

## Use SQL Magic to answer questions about the data
First create a connection to the database with sqlmagic.

In [3]:
# Connect SQL Magic to the database
%load_ext sql 
%sql sqlite:///CHICAGO.db


Find the total number of crimes recorded.


In [4]:
%sql SELECT COUNT(PRIMARY_TYPE) AS TOTAL_CRIME FROM CHICAGO_CRIME_DATA;


 * sqlite:///CHICAGO.db
Done.


TOTAL_CRIME
533


List community area names and numbers with per capita income less than 11000.

In [5]:
%sql SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER, PER_CAPITA_INCOME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000;

 * sqlite:///CHICAGO.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER,PER_CAPITA_INCOME
West Garfield Park,26.0,10934
South Lawndale,30.0,10402
Fuller Park,37.0,10432
Riverdale,54.0,8201


List all case numbers for crimes involving minors (children are not considered minors for the purposes of crime analysis)

In [6]:
%sql SELECT CASE_NUMBER FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%CHILD%'

 * sqlite:///CHICAGO.db
Done.


CASE_NUMBER
HN567387
HR391350
HN144152


List all kidnapping crimes involving a child

In [7]:
%sql SELECT * FROM CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE LIKE '%KIDNAP%' AND DESCRIPTION LIKE '%CHILD%'

 * sqlite:///CHICAGO.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


List the kind of crimes that were recorded at schools

In [8]:
%sql SELECT DISTINCT PRIMARY_TYPE, LOCATION_DESCRIPTION FROM CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * sqlite:///CHICAGO.db
Done.


PRIMARY_TYPE,LOCATION_DESCRIPTION
BATTERY,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, BUILDING"
ASSAULT,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS"
PUBLIC PEACE VIOLATION,"SCHOOL, PRIVATE, BUILDING"
PUBLIC PEACE VIOLATION,"SCHOOL, PUBLIC, BUILDING"


List the type of schools along with the average safety score for each type.

In [9]:
%sql SELECT "Elementary, Middle, or High School" AS SCHOOL_TYPE, AVG(SAFETY_SCORE) AS AVG_S_SCORE FROM CHICAGO_PUBLIC_SCHOOLS GROUP BY SCHOOL_TYPE

 * sqlite:///CHICAGO.db
Done.


SCHOOL_TYPE,AVG_S_SCORE
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


List 5 community areas with highest % of households below poverty line.

In [10]:
%sql SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5

 * sqlite:///CHICAGO.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


Which community area is most crime prone? Display the coumminty area number only.

In [11]:
%sql SELECT COMMUNITY_AREA_NUMBER, COUNT(PRIMARY_TYPE) FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(PRIMARY_TYPE) DESC LIMIT 1

 * sqlite:///CHICAGO.db
Done.


COMMUNITY_AREA_NUMBER,COUNT(PRIMARY_TYPE)
25.0,43


Use a sub-query to find the name of the community area with highest hardship index

In [12]:
%sql SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE HARDSHIP_INDEX IN (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA)

 * sqlite:///CHICAGO.db
Done.


COMMUNITY_AREA_NAME
Riverdale


Use a sub-query to determine the Community Area Name with most number of crimes?

In [13]:
%%sql
SELECT CD.COMMUNITY_AREA_NAME
FROM CENSUS_DATA AS CD
INNER JOIN CHICAGO_CRIME_DATA AS CCD ON CCD.COMMUNITY_AREA_NUMBER = CD.COMMUNITY_AREA_NUMBER
GROUP BY CD.COMMUNITY_AREA_NAME
ORDER BY COUNT(CCD.PRIMARY_TYPE) DESC
LIMIT 1

 * sqlite:///CHICAGO.db
Done.


COMMUNITY_AREA_NAME
Austin


## Using Joins to Answer Questions:
1. Write and execute a SQL query to list the school names, community names and average attendance for communities with a hardship index of 98.
2. Write and execute a SQL query to list all crimes that took place at a school. Include case number, crime type and community name.

In [14]:
%%sql
SELECT 
    CPS.NAME_OF_SCHOOL, CD.COMMUNITY_AREA_NAME, CPS.AVERAGE_STUDENT_ATTENDANCE
FROM CHICAGO_PUBLIC_SCHOOLS AS CPS
LEFT JOIN CENSUS_DATA AS CD 
ON CD.COMMUNITY_AREA_NUMBER = CPS.COMMUNITY_AREA_NUMBER
WHERE CD.HARDSHIP_INDEX = 98


 * sqlite:///CHICAGO.db
Done.


NAME_OF_SCHOOL,COMMUNITY_AREA_NAME,AVERAGE_STUDENT_ATTENDANCE
George Washington Carver Military Academy High School,Riverdale,91.60%
George Washington Carver Primary School,Riverdale,90.90%
Ira F Aldridge Elementary School,Riverdale,92.90%
William E B Dubois Elementary School,Riverdale,93.30%


In [15]:
%%sql 
SELECT 
    CCD.CASE_NUMBER, CCD.PRIMARY_TYPE, CD.COMMUNITY_AREA_NAME
FROM CHICAGO_CRIME_DATA AS CCD 
LEFT JOIN CENSUS_DATA AS CD 
ON CCD.COMMUNITY_AREA_NUMBER = CD.COMMUNITY_AREA_NUMBER
WHERE CCD.LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * sqlite:///CHICAGO.db
Done.


CASE_NUMBER,PRIMARY_TYPE,COMMUNITY_AREA_NAME
HL353697,BATTERY,South Shore
HL725506,BATTERY,Lincoln Square
HP716225,BATTERY,Douglas
HH639427,BATTERY,Austin
JA460432,BATTERY,Ashburn
HS200939,CRIMINAL DAMAGE,Austin
HK577020,NARCOTICS,Rogers Park
HS305355,NARCOTICS,Brighton Park
HT315369,ASSAULT,East Garfield Park
HR585012,CRIMINAL TRESPASS,Ashburn


## Create a View Using New Column Names 
- Create a View for Public Schools Data
- Execute a SQL statement that returns all columns from the view
- Execute a SQL statement that returns the school names and the leader's ratings from the view 

In [16]:
%%sql
CREATE VIEW IF NOT EXISTS PUBLIC_SCHOOL_VIEW AS 
SELECT
    NAME_OF_SCHOOL AS SCHOOL_NAME, SAFETY_ICON AS SAFETY_RATING, FAMILY_INVOLVEMENT_ICON AS FAMILY_RATING,
    ENVIRONMENT_ICON AS ENVIRONMENT_RATING, INSTRUCTION_ICON AS INSTRUCTION_RATING,
    LEADERS_ICON AS LEADERSHIP_RATING, TEACHERS_ICON AS TEACHERS_RATING
FROM CHICAGO_PUBLIC_SCHOOLS;

PRAGMA table_info(PUBLIC_SCHOOL_VIEW);



 * sqlite:///CHICAGO.db
Done.
Done.


cid,name,type,notnull,dflt_value,pk
0,SCHOOL_NAME,TEXT,0,None,0
1,SAFETY_RATING,TEXT,0,None,0
2,FAMILY_RATING,TEXT,0,None,0
3,ENVIRONMENT_RATING,TEXT,0,None,0
4,INSTRUCTION_RATING,TEXT,0,None,0
5,LEADERSHIP_RATING,TEXT,0,None,0
6,TEACHERS_RATING,TEXT,0,None,0


In [17]:
%%sql
SELECT 
    SCHOOL_NAME, LEADERSHIP_RATING
FROM PUBLIC_SCHOOL_VIEW
LIMIT 5

 * sqlite:///CHICAGO.db
Done.


SCHOOL_NAME,LEADERSHIP_RATING
Abraham Lincoln Elementary School,Weak
Adam Clayton Powell Paideia Community Academy Elementary School,Weak
Adlai E Stevenson Elementary School,Weak
Agustin Lara Elementary Academy,Weak
Air Force Academy High School,Weak


## Create A Stored Procedure and Transaction
- Use school_id and leader_score as parameters
- Update the leader_score in the public school table to the value identified by the leader_score parameter
- Use an IF statement to update the leaders_icon field in the public table for the school identified by the school id
- Test the query for the leader score
- Rollback to previous values if the leaders_score is not in the CASE statement
- Commit the current updates id everything goes find

In [18]:
%%sql 
SELECT * FROM CHICAGO_PUBLIC_SCHOOLS LIMIT 5

 * sqlite:///CHICAGO.db
Done.


School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,Collaborative_Name,Adequate_Yearly_Progress_Made_,Track_Schedule,CPS_Performance_Policy_Status,CPS_Performance_Policy_Level,HEALTHY_SCHOOL_CERTIFIED,Safety_Icon,SAFETY_SCORE,Family_Involvement_Icon,Family_Involvement_Score,Environment_Icon,Environment_Score,Instruction_Icon,Instruction_Score,Leaders_Icon,Leaders_Score,Teachers_Icon,Teachers_Score,Parent_Engagement_Icon,Parent_Engagement_Score,Parent_Environment_Icon,Parent_Environment_Score,AVERAGE_STUDENT_ATTENDANCE,Rate_of_Misconducts__per_100_students_,Average_Teacher_Attendance,Individualized_Education_Program_Compliance_Rate,Pk_2_Literacy__,Pk_2_Math__,Gr3_5_Grade_Level_Math__,Gr3_5_Grade_Level_Read__,Gr3_5_Keep_Pace_Read__,Gr3_5_Keep_Pace_Math__,Gr6_8_Grade_Level_Math__,Gr6_8_Grade_Level_Read__,Gr6_8_Keep_Pace_Math_,Gr6_8_Keep_Pace_Read__,Gr_8_Explore_Math__,Gr_8_Explore_Read__,ISAT_Exceeding_Math__,ISAT_Exceeding_Reading__,ISAT_Value_Add_Math,ISAT_Value_Add_Read,ISAT_Value_Add_Color_Math,ISAT_Value_Add_Color_Read,Students_Taking__Algebra__,Students_Passing__Algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),Net_Change_EXPLORE_and_PLAN,11th Grade Average ACT (2011),Net_Change_PLAN_and_ACT,College_Eligibility__,Graduation_Rate__,College_Enrollment_Rate__,COLLEGE_ENROLLMENT,General_Services_Route,Freshman_on_Track_Rate__,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police_District,Location
610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf,Fullerton Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,Yes,Very Strong,99.0,Very Strong,99,Strong,74.0,Strong,66.0,Weak,65,Strong,70,Strong,56,Average,47,96.00%,2.0,96.40%,95.80%,80.1,43.3,89.6,84.9,60.7,62.6,81.9,85.2,52,62.4,66.3,77.9,69.7,64.4,0.2,0.9,Yellow,Green,67.1,54.5,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,813,33,NDA,1171699.458,1915829.428,41.92449696,-87.64452163,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
610281,Adam Clayton Powell Paideia Community Academy Elementary School,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610281.pdf,Skyway Elementary Network,SOUTH SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,54.0,Strong,66,Strong,74.0,Very Strong,84.0,Weak,63,Strong,76,Weak,46,Average,50,95.60%,15.7,95.30%,100.00%,62.4,51.7,21.9,15.1,29,42.8,38.5,27.4,44.8,42.7,14.1,34.4,16.8,16.5,0.7,1.4,Green,Green,17.2,27.3,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,521,46,NDA,1196129.985,1856209.466,41.76032435,-87.55673627,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610185.pdf,Midway Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 2,No,Strong,61.0,NDA,NDA,Average,50.0,Weak,36.0,Weak,NDA,NDA,NDA,Average,47,Weak,41,95.70%,2.3,94.70%,98.30%,53.7,26.6,38.3,34.7,43.7,57.3,48.8,39.2,46.8,44,7.5,21.9,18.3,15.5,-0.9,-1.0,Red,Red,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,1324,44,NDA,1148427.165,1851012.215,41.74711093,-87.73170248,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"
609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,60609,(773) 535-4389,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609993.pdf,Pershing Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,56.0,Average,44,Average,45.0,Weak,37.0,Weak,65,Average,48,Average,53,Strong,58,95.50%,10.4,95.80%,100.00%,76.9,NDA,26,24.7,61.8,49.7,39.2,27.2,69.7,60.6,9.1,18.2,11.1,9.6,0.9,2.4,Green,Green,42.9,25,NDA,N

In [19]:
# Python Function to Update the Leader Score 
def update_leader_score(school_id, leader_score):
    try: 
        test_con = sqlite3.connect('CHICAGO.db')
        test_cursor = test_con.cursor()

        test_con.isolation_level = None  # Turn off Auto-commit 

        # Start the transaction
        test_con.execute("BEGIN TRANSACTION")

        # Update the leader score
        test_cursor.execute("UPDATE CHICAGO_PUBLIC_SCHOOLS SET Leaders_Score = ? WHERE School_id = ?", (leader_score, school_id))

        # Update Leaders Icon 
        test_cursor.execute("""
            UPDATE CHICAGO_PUBLIC_SCHOOLS 
            SET Leaders_Icon = CASE
                    WHEN ? >= 80 AND ? <= 99 THEN 'Very strong'
                    WHEN ? >= 60 AND ? <= 79 THEN 'Strong'
                    WHEN ? >= 40 AND ? <= 59 THEN 'Average'
                    WHEN ? >= 20 AND ? <= 39 THEN 'Weak'
                    WHEN ? >= 0 AND ? <= 19 THEN 'Very Weak'
                    ELSE Leaders_Icon  -- Keeps the value
            END
            WHERE School_ID = ?
                """, (leader_score, leader_score, leader_score, leader_score, leader_score, leader_score, leader_score, leader_score, leader_score, leader_score, school_id))
        
        # Check if the Leaders_Icon was actually updated
        test_cursor.execute("SELECT changes()")
        rows_changed = test_cursor.fetchone()[0]

        if rows_changed == 0:  # No update means the score was out of range
            test_con.execute("ROLLBACK")
            print(f"Leader score {leader_score} is not in a valid range. Rollback performed.")
            return  # Exit the function early
        
        test_con.commit()
        print(f"School with id {school_id} updated the leader_score to {leader_score}")
    except sqlite3.Error as e:
        print(f'Problem connecting to the database: {e}')
    finally:
        if test_con:
            test_con.close()


test_the_function = update_leader_score(610038, 50)
print(test_the_function)
invalid_score = update_leader_score(609993, 101)


School with id 610038 updated the leader_score to 50
None
School with id 609993 updated the leader_score to 101


In [20]:
%%sql 
SELECT SCHOOL_ID, LEADERS_SCORE, LEADERS_ICON
FROM CHICAGO_PUBLIC_SCHOOLS
WHERE SCHOOL_ID = 610038;

SELECT SCHOOL_ID, LEADERS_SCORE, LEADERS_ICON
FROM CHICAGO_PUBLIC_SCHOOLS
WHERE SCHOOL_ID = 609993;

 * sqlite:///CHICAGO.db
Done.
Done.


School_ID,Leaders_Score,Leaders_Icon
609993,101,Weak
